In [1]:
# Import required Python libraries
import time
import base64
import hashlib
import hmac
import urllib.request
import json
import websocket
from websocket import create_connection
import requests
from dotenv import load_dotenv
import os
import _thread
import sys


load_dotenv()

True

# Fetch Websocket Authentication Token via Rest

In [2]:
# This code cell was taken directly from the kraken documentation shown here:
# https://support.kraken.com/hc/en-us/articles/360034437672-How-to-retrieve-a-WebSocket-authentication-token-Example-code-in-Python-3

# Copy/paste API public key and API private key as displayed in account management
api_key = os.getenv('API_KEY')
api_secret = os.getenv('PRIVATE_KEY')
 
# Variables (API method, nonce, and POST data)
api_path = '/0/private/GetWebSocketsToken'
api_nonce = str(int(time.time()*1000))
api_post = 'nonce=' + api_nonce
 
# Cryptographic hash algorithms
api_sha256 = hashlib.sha256(api_nonce.encode('utf-8') + api_post.encode('utf-8'))
api_hmac = hmac.new(base64.b64decode(api_secret), api_path.encode('utf-8') + api_sha256.digest(), hashlib.sha512)
 
# Encode signature into base64 format used in API-Sign value
api_signature = base64.b64encode(api_hmac.digest())
 
# HTTP request (POST)
api_request = urllib.request.Request('https://api.kraken.com/0/private/GetWebSocketsToken', api_post.encode('utf-8'))
api_request.add_header('API-Key', api_key)
api_request.add_header('API-Sign', api_signature)
api_response = urllib.request.urlopen(api_request).read().decode()

# Output API response
ws_token = json.loads(api_response)['result']['token']

# Stream Trade Prices

In [ ]:
command = {
  "event": "subscribe",
  "pair": [
    "XBT/USD"
  ],
  "subscription": {
    "name": "trade"
  }
}

data = ""
    
# Define WebSocket callback functions
def ws_thread(*args):
    ws = websocket.WebSocketApp('wss://ws.kraken.com/', on_open=ws_open, on_message=ws_message)
    ws.run_forever()
    
    
def ws_open(ws):
    ws.send(json.dumps(command).encode('utf-8'))

def ws_message(ws, ws_data):
    data = json.loads(ws_data)
    if 'event' not in data:
        print(data)

# Start new thread for WebSocket interface
_thread.start_new_thread(ws_thread, ())


# Just run for a minute for now 
# TODO: fix this
try:
    time.sleep(1)
except KeyboardInterrupt:
    sys.exit(0)

# Stream OHLC Data

In [ ]:
command = {
  "event": "subscribe",
  "pair": [
    "XBT/EUR"
  ],
  "subscription": {
    "interval": 5,
    "name": "ohlc"
  }
}


data = ""
    
# Define WebSocket callback functions
def ws_thread(*args):
    ws = websocket.WebSocketApp('wss://ws.kraken.com/', on_open=ws_open, on_message=ws_message)
    ws.run_forever()
    
    
def ws_open(ws):
    ws.send(json.dumps(command).encode('utf-8'))

def ws_message(ws, ws_data):
    data = json.loads(ws_data)
    if 'event' not in data:
        print(data)

# Start new thread for WebSocket interface
_thread.start_new_thread(ws_thread, ())


# Just run for a minute for now 
# TODO: fix this
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    sys.exit(0)

# Stream Spread

In [ ]:
command = {
  "event": "subscribe",
  "pair": [
    "XBT/EUR"
  ],
  "subscription": {
    "name":"spread"
  }
}


data = ""
    
# Define WebSocket callback functions
def ws_thread(*args):
    ws = websocket.WebSocketApp('wss://ws.kraken.com/', on_open=ws_open, on_message=ws_message)
    ws.run_forever()
    
    
def ws_open(ws):
    ws.send(json.dumps(command).encode('utf-8'))

def ws_message(ws, ws_data):
    data = json.loads(ws_data)
    if 'event' not in data:
        print(data)

# Start new thread for WebSocket interface
_thread.start_new_thread(ws_thread, ())


# Just run for a minute for now 
# TODO: fix this
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    sys.exit(0)

[357, ['39883.80000', '39883.90000', '1704480568.015058', '0.95659115', '6.42661672'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480581.874749', '0.95659115', '6.43914798'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.704949', '1.87290348', '6.43914798'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.733113', '1.87290348', '4.83665155'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.733212', '1.87290348', '3.23415512'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.733618', '1.87290348', '1.63165869'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.735534', '1.87290348', '0.02916226'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.751340', '2.11290348', '0.02916226'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39883.90000', '1704480582.759166', '2.11290348', '0.01663100'], 'spread', 'XBT/EUR']
[357, ['39883.80000', '39884

# Stream Order Book Data

In [ ]:
# https://support.kraken.com/hc/en-us/articles/4410463626132-Example-code-for-interpreting-the-WebSocket-API-v1-order-book-feed-asynchronous-multi-threaded-
# Define order book variables

api_depth = 5
api_symbol = 'XBT/USD' 
api_book = {'bid':{}, 'ask':{}}

# Define order book update functions
def dicttofloat(data):
    return float(data[0])

def api_book_update(api_book_side, api_book_data):
    for data in api_book_data:
        price_level = data[0]
        volume = data[1]
        if float(volume) > 0.0:
            api_book[api_book_side][price_level] = volume
        else:
            api_book[api_book_side].pop(price_level)
        if api_book_side == 'bid':
            api_book['bid'] = dict(sorted(api_book['bid'].items(), key=dicttofloat, reverse=True)[:api_depth])
        elif api_book_side == 'ask':
            api_book['ask'] = dict(sorted(api_book['ask'].items(), key=dicttofloat)[:api_depth])

# Define WebSocket callback functions
def ws_thread(*args):
    ws = websocket.WebSocketApp('wss://ws.kraken.com/', on_open=ws_open, on_message=ws_message)
    ws.run_forever()

def ws_open(ws):
    print('sending message..')
    ws.send('{"event":"subscribe", "subscription":{"name":"book", "depth":%(api_depth)d}, "pair":["%(api_symbol)s"]}' % {'api_depth':api_depth, 'api_symbol':api_symbol})

def ws_message(ws, ws_data):
    print('response received')
    api_data = json.loads(ws_data)
    if 'event' in api_data:
        return
    else:
        if 'as' in api_data[1]:
            api_book_update('ask', api_data[1]['as'])
            api_book_update('bid', api_data[1]['bs'])
        else:
            for data in api_data[1:len(api_data)-2]:
                if 'a' in data:
                    api_book_update('ask', data['a'])
                elif 'b' in data:
                    api_book_update('bid', data['b'])

# Start new thread for WebSocket interface
_thread.start_new_thread(ws_thread, ())

# Output order book (once per second) in main thread
try:
    while True:
        if len(api_book['bid']) < api_depth or len(api_book['ask']) < api_depth:
            time.sleep(1)
        else:
            bid = sorted(api_book['bid'].items(), key=dicttofloat, reverse=True)
            ask = sorted(api_book['ask'].items(), key=dicttofloat)
            print('Bid\t\t\t\t\t\t\tAsk')
            for count in range(api_depth):
                print('%(bidprice)s (%(bidvolume)s)\t\t\t\t%(askprice)s (%(askvolume)s)' \
                      % {'bidprice':bid[count][0], 'bidvolume':bid[count][1], 'askprice':ask[count][0], 'askvolume':ask[count][1]})
            time.sleep(1)
except KeyboardInterrupt:
    sys.exit(0)

In [ ]:
api_depth = 5
api_symbol = 'XBT/USD' 

'{"event":"subscribe", "subscription":{"name":"book", "depth":%(api_depth)d}, "pair":["%(api_symbol)s"]}' % {'api_depth':api_depth, 'api_symbol':api_symbol}